<a href="https://colab.research.google.com/github/GarnetKangSB/TIL/blob/master/0721_DocumentSurmmarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import math

## Class 화

class TextRankBySentence():

    def __init__(self, sentences, alpha_rate = 0.85, update_count = 3):
        self.alpha_rate = alpha_rate
        self.iteration = 0
        self.sentences = sentences
        ### 문장 전처리
        # if 필요하면
        # 문장으로 쪼갬
        if len(sentences) == 1:
            try:
                print("<< Sentence Split >>")
                self.sentences = sentences[0].split(". ")
                self.sentences_len = len(self.sentences)
                
            except:
                print("<< Input Error >>")
                exit()
                pass
        else:
            self.sentences_len = len(sentences)
        
        # 문장(=토큰,그래프 노드)끼리의 연결(=유사도,그래프 엣지)을 0으로 초기화
        self.similarity = np.zeros((self.sentences_len,self.sentences_len),dtype=np.float32)
        # 문장 간 유사도 계산
        self.set_similarity()
        # 문장 간 유사도 출력
        self.show_similarity()
        

        ### 그래프 초기화

        # 토큰(그래프 edge)별로 Matrix 형태로 가중치 생성 후 0으로 초기화
        self.weighted_edge = np.zeros((self.sentences_len,self.sentences_len),dtype=np.float32)
        # 토큰(그래프 edge)별로 Vector 형태로 스코어 생성 후 1으로 초기화
        self.score = np.ones((self.sentences_len),dtype=np.float32)

        # 유사도
        self.tmp_similarity = self.similarity.copy()
        # 이전 스코어 저장
        self.prev_score = self.score.copy()
        # 유사도를 토대로 스코어 계산
        for i_idx in range(self.sentences_len):
            self.score[i_idx] = np.sum(self.similarity[i_idx][:])
        print("<< Initial Score >>")
        print(self.score)
        # 유사도를 토대로 가중치 계산
        for i_idx in range(self.sentences_len):
            for j_idx in range(self.sentences_len):  
                self.weighted_edge[i_idx][j_idx] = ( self.prev_score[i_idx] / self.score[i_idx] * self.tmp_similarity[i_idx][j_idx] )

        self.update(update_count)


    def set_similarity(self):
        # 글에서 문장으로 나눈 리스트의
        # 원소쌍을 유사도 계산 함수에 입력함
        for i_idx in range(self.sentences_len-1):
            for j_idx in range(i_idx + 1, self.sentences_len):
                # 대각선 대칭으로 저장
                self.similarity[j_idx][i_idx] = self.similarity[i_idx][j_idx] = self.check_similarity(self.sentences[i_idx], self.sentences[j_idx])
        

    # 문장 간 유사도 계산 함수 #####
    # 입력 : 문장 2개 # 출력 : 유사도
    def check_similarity(self, sentence1, sentence2):

        # 토큰화를 수행합니다.
        token_s1 = sentence1.split()
        token_s2 = sentence2.split()

        # 토큰 합집합 생성
        union = set(token_s1).union(set(token_s2))

        # 토큰 교집합 생성
        intersection = set(token_s1).intersection(set(token_s2))

        # 유사도 계산 후 반환
        try:
            return len(intersection)/len(union) # n(교집합/합집합)
        except:
            return 0 # case: 합집합={}

    # 그래프 노드 스코어 갱신 함수 #####
    def cal_score(self):
        # 이전 스코어 기록
        self.prev_score = self.score.copy()
        # 유사도를 토대로 스코어 계산
        for i_idx in range(self.sentences_len):
            self.score[i_idx] = (1-self.alpha_rate) + self.alpha_rate * np.sum(self.tmp_similarity[i_idx][:])
            for j_idx in range(self.sentences_len):  
                self.tmp_similarity[i_idx][j_idx] = self.tmp_similarity[i_idx][j_idx] * (self.score[i_idx] / self.prev_score[i_idx])
        # 현재 스코어 출력
        self.show_score()

    # 그래프 엣지 가중치 갱신 함수 #####
    def cal_weight(self):
        # 유사도를 토대로 가중치 계산
        for i_idx in range(self.sentences_len):
            for j_idx in range(self.sentences_len):  
                self.weighted_edge[i_idx][j_idx] = ( self.prev_score[i_idx] / self.score[i_idx] * self.tmp_similarity[i_idx][j_idx] )        
        # 현재 가중치 출력
        self.show_weight()

    # 그래프 갱신 #####
    def update(self, update_count):
        for i in range(update_count):
            self.iteration += 1
            self.cal_score()
            self.cal_weight()
    
    # 문장 간 유사도 출력 함수 #####
    def show_similarity(self):
        print("<< Similarity >>")
        print(self.similarity)

    # 그래프 노드 스코어 출력 함수 #####
    def show_score(self):
        print("<< {:2}th Node Score >>".format(self.iteration))
        print(self.score,end="\n\n")

    # 그래프 엣지 가중치 출력 함수 #####
    def show_weight(self):
        print("<< {:2}th Edge Weight >>".format(self.iteration))
        print(self.weighted_edge,end="\n\n")

    # 최중요 문장 출력 함수 #####
    def show_important_sentence(self):
        # score에서 최댓값을 찾아서
        # 그 최댓값의 Index를 찾아서
        # 그 Index로 문장을 출력
        print(self.sentences[list(self.score).index(max(self.score))])

# 테스트 #####
docs = ["딸기 바나나 사과 파인애플 수박","바나나 사과 딸기 포도", "복숭아 수박","파인애플 사과 딸기 바나나"]
test = TextRankBySentence(docs)


docs = ["딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나"]
test = TextRankBySentence(docs)

test.show_important_sentence()

<< Similarity >>
[[0.         0.5        0.16666667 0.8       ]
 [0.5        0.         0.         0.6       ]
 [0.16666667 0.         0.         0.        ]
 [0.8        0.6        0.         0.        ]]
<< Initial Score >>
[1.4666667  1.1        0.16666667 1.4000001 ]
<<  1th Node Score >>
[1.3966666  1.085      0.29166666 1.34      ]

<<  1th Edge Weight >>
[[0.         0.5        0.16666667 0.8       ]
 [0.49999997 0.         0.         0.6       ]
 [0.16666667 0.         0.         0.        ]
 [0.8000001  0.6000001  0.         0.        ]]

<<  2th Node Score >>
[1.3371667  1.07225    0.39791664 1.289     ]

<<  2th Edge Weight >>
[[0.         0.47613633 0.15871212 0.7618181 ]
 [0.4931818  0.         0.         0.5918182 ]
 [0.29166666 0.         0.         0.        ]
 [0.7657143  0.57428575 0.         0.        ]]

<<  3th Node Score >>
[1.2865916  1.0614125  0.48822916 1.24565   ]

<<  3th Edge Weight >>
[[0.         0.45585227 0.15195075 0.72936356]
 [0.48738638 0.         0